<a href="https://colab.research.google.com/github/antopeza/Final-Project-Brainster/blob/main/NEW_code_Doc2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from numpy import random
from numpy import vstack
import matplotlib.pyplot as plt
import os 

import re

import string
from string import punctuation

from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import xgboost as xgb

import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, word_tokenize

import gensim
from gensim.parsing.preprocessing import remove_stopwords

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Embedding, LSTM, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

from imblearn.over_sampling import SMOTE
from collections import Counter

#from scikeras.wrappers import KerasClassifier

import pickle
from pickle import dump
from pickle import load

from imblearn.over_sampling import SMOTE
from collections import Counter

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive 
drive.mount('/drive')

Mounted at /drive


In [3]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
stops = set(stopwords.words('english'))
#print(stops)

In [5]:
stemmer =  PorterStemmer()
lemmer = WordNetLemmatizer()

In [6]:
#df_raw = pd.read_csv(path)
df_raw = pd.read_csv('/drive/My Drive/MACHINE LEARNING/FINAL PROJECT/train.csv')

In [ ]:
print('shape of df  = ', df_raw.shape)
print()
df_raw.head(2)

shape of df  =  (404290, 6)



,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0


In [ ]:
df_raw[df_raw.isnull().any(axis=1)]

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0
363362,363362,493340,493341,NaN,My Chinese name is Haichao Yu. What English na...,0


In [8]:
df_raw = df_raw.fillna("")
df_raw.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [9]:
# randomly selected samples from df for the new df to perform operations
df = df_raw.sample(5000)
df = df[['question1', 'question2', 'is_duplicate']]
print(df.shape)

(5000, 3)


In [ ]:
df_copy = df.copy()

# Clean Data

In [10]:
def get_wordnet_pos(tag):
    if tag.startswith("N"):
        return wordnet.NOUN
    if tag.startswith("J"):
        return wordnet.ADJ
    if tag.startswith("V"):
        return wordnet.VERB
    if tag.startswith("R"):
        return wordnet.ADV
    return wordnet.NOUN

def clean_sentence(sentence, stopwords = False, lemmatize = False, stem = False):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)  # removing all caracters that are not alpha numeric
    #sentence = re.sub(r'\s{2,}', ' ', sentence
   
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    
    if stopwords:
        sentence = remove_stopwords(sentence)  
        
    if lemmatize:
        nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
        wn_tagged = map(lambda x: (x[0], get_wordnet_pos(x[1])), nltk_tagged)
        res_words = []
        for word, tag in wn_tagged:
            if tag is None:
                res_words.append(word)
            else :
                res_words.append(lemmer.lemmatize(word, tag))
        sentence = " ".join(res_words)
        
    if stem:
        sent_stemmed = ''
        for word in sentence.split():
            sent_stemmed += ' '+ stemmer.stem(word)
        sentence = sent_stemmed
   
    return sentence

def get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False):
    #sents1 = dfq[['question1']]  
    #sents2 = dfq[['question2']]
    cleaned_sentences1 = []
    cleaned_sentences2 = []
    
    for index, row in df.iterrows():
        # print (index, row)
        cleaned1 = clean_sentence(row['question1'], stopwords, lemmatize, stem) 
        cleaned2 = clean_sentence(row['question2'], stopwords, lemmatize, stem)

        cleaned_sentences1.append(cleaned1)
        cleaned_sentences2.append(cleaned2)
    return cleaned_sentences1, cleaned_sentences2

In [11]:
def to_df(q1cleaned, q2cleaned, df):
    '''cleaned sentences to df'''
    X_temp1 = pd.DataFrame()
    X_temp2 = pd.DataFrame()
    X_temp1['q1'] = pd.DataFrame(q1cleaned, index = df.index)
    X_temp2['q2'] = pd.DataFrame(q2cleaned, index = df.index)
    X_temp = pd.concat([X_temp1, X_temp2], axis = 1)
    X_temp['is_duplicate'] = df_raw['is_duplicate']
    return X_temp

#### With Stopwords, lemmatized

In [ ]:
# With stopwords lemmatized, not stemmed
q1_sw_lemma, q2_sw_lemma = get_cleaned_senteces(df, stopwords = False, lemmatize = True, stem = False)
assert len(q1_sw_lemma) == len(q2_sw_lemma)
df_sw_lemma = to_df(q1_sw_lemma, q2_sw_lemma, df)
df_sw_lemma.shape

(5000, 3)

#### With Stopwords, stemmed

In [12]:
q1_sw_stem, q2_sw_stem = get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = True)
assert len(q1_sw_stem) == len(q2_sw_stem)
df_sw_stem = to_df(q1_sw_stem, q2_sw_stem, df)
df_sw_stem.shape

(5000, 3)

#### With stopwords, not lemmatized nor stemmed

In [ ]:
q1_sw, q2_sw = get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False)
assert len(q1_sw) == len(q2_sw)
df_sw = to_df(q1_sw, q2_sw, df)
df_sw.shape

(5000, 3)

#### Without Stopwords, lemmatized


In [ ]:
q1_lemma, q2_lemma = get_cleaned_senteces(df, stopwords = True, lemmatize = True, stem = False)
assert len(q1_lemma) == len(q2_lemma)
df_lemma = to_df(q1_lemma, q2_lemma, df)
df_lemma.shape

(5000, 3)

#### Without Stopwords, stemmed

In [13]:
q1_stem, q2_stem = get_cleaned_senteces(df, stopwords = True, lemmatize = False, stem = True)
assert len(q1_stem) == len(q2_stem)
df_stem = to_df(q1_stem, q2_stem, df)
df_stem.shape

(5000, 3)

#### Without Stopwords, not lemmatized nor stemmed

In [ ]:
q1_out, q2_out = get_cleaned_senteces(df, stopwords = True, lemmatize = False, stem = False)
assert len(q1_out) == len(q2_out)
df_out = to_df(q1_out, q2_out, df)
df_out.shape

(5000, 3)

In [ ]:
display ('With stopwords, without lemma and stem', df_sw.iloc[[22]])
display ('With stopwords, lemmatized and without stem', df_sw_lemma.iloc[[22]])
display ('With stopwords, stemmed and without lemma', df_sw_stem.iloc[[22]])
display ('Without stopwords, lemma and stem', df_out.iloc[[22]])
display ('Without stopwords and stem, lemmatized', df_lemma.iloc[[22]])
display ('Without stopwords and lemma, stemmed', df_stem.iloc[[22]])

'With stopwords, without lemma and stem'

,q1,q2,is_duplicate
272872,what does it feel like to be on quora,what does it feel like to post on quora,1


'With stopwords, lemmatized and without stem'

,q1,q2,is_duplicate
272872,what do it feel like to be on quora,what do it feel like to post on quora,1


'With stopwords, stemmed and without lemma'

...

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


'Without stopwords and stem, lemmatized'

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


'Without stopwords and lemma, stemmed'

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


In [ ]:
# print('Value counts:\n', X_sw_train['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_train['is_duplicate'].value_counts()/X_sw_train['is_duplicate'].count()*100)
# X_sw_train['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of train labels', color = 'purple', size = 12)
# plt.show()

In [ ]:
# print('Value counts:\n', X_sw_test['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_test['is_duplicate'].value_counts()/X_sw_test['is_duplicate'].count()*100)
# X_sw_test['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of test labels', color = 'purple', size = 12)
# plt.show()

In [ ]:
# display (X_sw_train.head(2))
# display(X_sw_test.head(2))
# display(df_clean.head(2))

In [ ]:
#X_sw_test[X_sw_test.isnull().any(axis=1)]

# *TOKENIZATION

In [14]:
def tokenization(text):
    tokens = re.split(' ',text)
    return tokens
def token(text):
    text = text.apply(lambda x: tokenization(x))
    return text

In [ ]:
#Tokenize - lemmatized data without stopwords
df_lemma['q1'] = df_lemma['q1'].pipe(token) 
df_lemma['q2'] = df_lemma['q2'].pipe(token) 
df_lemma.head(2)

,q1,q2,is_duplicate
296968,"[indian, lamborghini, usa, legally]","[study, usa, indian, good, idea]",0
236824,"[learn, english]","[learn, english]",0


In [ ]:
# Tokenize - lemmatized data with stopwords
df_sw_lemma['q1'] = df_sw_lemma['q1'].pipe(token) 
df_sw_lemma['q2'] = df_sw_lemma['q2'].pipe(token) 
df_sw_lemma.head(2)

In [15]:
# Tokenize - Stemmed data with stopwords
df_sw_stem['q1'] = df_sw_stem['q1'].pipe(token) 
df_sw_stem['q2'] = df_sw_stem['q2'].pipe(token) 
df_sw_stem.head(2)

,q1,q2,is_duplicate
11965,"[, what, is, the, list, of, job, titl, at, rsm...","[, which, countri, is, the, best, to, work, in...",0
342404,"[, should, peopl, over, 94, not, be, allow, to...","[, should, peopl, over, 97, not, be, allow, to...",1


In [ ]:
# Tokenize - Stemmed data without stopwords
df_stem['q1'] = df_stem['q1'].pipe(token) 
df_stem['q2'] = df_stem['q2'].pipe(token) 
df_stem.head(2)

,q1,q2,is_duplicate
327246,"[, can, i, regist, my, startup, compani, in, t...","[, can, i, start, a, compani, in, the, us, wit...",1
26328,"[, what, is, a, good, entrylevel, job, in, the...","[, what, are, the, best, entri, level, job, in...",0


In [ ]:
# Tokenize - With stopwords, not lemmatized nor stemmed
df_sw['q1'] = df_sw['q1'].pipe(token) 
df_sw['q2'] = df_sw['q2'].pipe(token) 
df_stem.head(2)

In [ ]:
# Tokenize - Without stopwords, not lemmatized nor stemmed
df_out['q1'] = df_out['q1'].pipe(token) 
df_out['q2'] = df_out['q2'].pipe(token) 
df_out.head(2)

NameError: ignored

In [ ]:
df_sw_lemma.head(2)

,q1,q2,is_duplicate
82361,who invent wash machine,who invent the washing machine,1
12785,be hack a crime when someone do it to earn mon...,what do pakistani girl look for in a guy objec...,0


#### Doc2Vec Without Stopwords, not stemmed not lemmatized

In [16]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def doc2vec(df):
  q1 = [TaggedDocument(d, [i]) for i, d in enumerate(df['q1'])]
  q2 = [TaggedDocument(d, [i]) for i, d in enumerate(df['q2'])]
  return q1, q2

In [17]:
def model_doc2vec(q1, vector, epochs):
  q1 = Doc2Vec(q1, dm = 1, vector_size = vector, window = 3, min_count = 2, epochs = epochs)
  return q1

### Apply Doc2Vec

In [ ]:
#lemmatized, without stopwords
q1_lemma_d2v, q2_lemma_d2v = doc2vec(df_lemma)
assert len(q1_lemma_d2v) == len(q2_lemma_d2v)

In [ ]:
#lemmatized with stopwords
q1_sw_lemma_d2v, q2_sw_lemma_d2v = doc2vec(df_sw_lemma)
assert len(q1_sw_lemma_d2v) == len(q2_sw_lemma_d2v)

In [18]:
#Stemmed without stopwords
q1_stem_d2v, q2_stem_d2v = doc2vec(df_stem)
assert len(q1_stem_d2v) == len(q2_stem_d2v)

In [19]:
#Stemmed with stopwords
q1_sw_stem_d2v, q2_sw_stem_d2v = doc2vec(df_sw_stem)
assert len(q1_sw_stem_d2v) == len(q2_sw_stem_d2v)

In [ ]:
#With Stopwords, not lemmatized, not stemmed
q1_sw_d2v, q2_sw_d2v = doc2vec(df_sw)
assert len(q1_sw_d2v) == len(q2_sw_d2v)

In [ ]:
#Without stopwords, not lemmatized, not stemmed 
q1_out_d2v, q2_out_d2v = doc2vec(df_out)
assert len(q1_out_d2v) == len(q2_out_d2v)

### Activating Doc2Vec model

In [20]:
#PARAMETER FOR DOC2VEC MODEL 
vector = 100
epochs = 30

In [ ]:
#lemmatized, without stopwords
X_q1_lemma_d2v = model_doc2vec(q1_lemma_d2v, vector, epochs)
X_q2_lemma_d2v = model_doc2vec(q2_lemma_d2v, vector, epochs)

In [ ]:
#lemmatized, with stopwords
X_q1_sw_lemma_d2v = model_doc2vec(q1_sw_lemma_d2v, vector, epochs)
X_q2_sw_lemma_d2v = model_doc2vec(q2_sw_lemma_d2v, vector, epochs)

In [21]:
#Stemmed, without stopwords
X_q1_stem_d2v = model_doc2vec(q1_stem_d2v, vector,  epochs)
X_q2_stem_d2v = model_doc2vec(q2_stem_d2v, vector,  epochs)

In [22]:
#Stemmed, with stopwords
X_q1_sw_stem_d2v = model_doc2vec(q1_sw_stem_d2v, vector, epochs)
X_q2_sw_stem_d2v = model_doc2vec(q2_sw_stem_d2v, vector, epochs)

In [ ]:
#With Stopwords, not lemmatized, not stemmed
X_q1_sw_d2v = model_doc2vec(q1_sw_d2v, vector, epochs)
X_q2_sw_d2v = model_doc2vec(q2_sw_d2v, vector, epochs)

In [ ]:
#Without stopwords, not lemmatized, not stemmed 
X_q1_out_d2v = model_doc2vec(q1_out_d2v, vector, epochs)
X_q2_out_d2v = model_doc2vec(q2_out_d2v, vector, epochs)


#### Convert to array 

In [ ]:
# Lemmatized, without stopwords, not stemmed

size = len(df) - 1
arr_q1_lemma = X_q1_lemma_d2v[0] 

for i in range(size):
  arrT = X_q1_lemma_d2v[i+1]
  arr_q1_lemma = np.vstack([arr_q1_lemma, arrT])

arr_q2_lemma = X_q2_lemma_d2v[0] 

for i in range(size):
  arrT = X_q2_lemma_d2v[i+1]
  arr_q2_lemma = np.vstack([arr_q2_lemma, arrT])

assert arr_q1_lemma.shape == arr_q2_lemma.shape

#arr_q1_lemma = create_arrays(X_q1_lemma_d2v)
#arr_q2_lemma = create_arrays(X_q2_lemma_d2v)

In [ ]:
# Lemmatized, with stopwords, not stemmed

size = len(df) - 1
arr_q1_sw_lemma = X_q1_sw_lemma_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_lemma_d2v[i+1]
  arr_q1_sw_lemma = np.vstack([arr_q1_sw_lemma, arrT])

arr_q2_sw_lemma = X_q2_sw_lemma_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_lemma_d2v[i+1]
  arr_q2_sw_lemma = np.vstack([arr_q2_sw_lemma, arrT])

assert arr_q1_sw_lemma.shape == arr_q2_sw_lemma.shape

# arr_q1_sw_lemma = create_arrays(X_q1_sw_lemma_d2v)
# arr_q2_sw_lemma = create_arrays(X_q2_sw_lemma_d2v) 

In [23]:
# Stemmed, with stopwords, not lemmatized

size = len(df) - 1
arr_q1_stem = X_q1_stem_d2v[0] 

for i in range(size):
  arrT = X_q1_stem_d2v[i+1]
  arr_q1_stem = np.vstack([arr_q1_stem, arrT])

arr_q2_stem = X_q2_stem_d2v[0] 

for i in range(size):
  arrT = X_q2_stem_d2v[i+1]
  arr_q2_stem = np.vstack([arr_q2_stem, arrT])

assert arr_q1_stem.shape == arr_q2_stem.shape

# arr_q1_stem = create_arrays(X_q1_stem_d2v)
# arr_q2_stem = create_arrays(X_q2_stem_d2v)

In [24]:
# Stemmed, without stopwords, not lemmatized 

size = len(df) - 1
arr_q1_sw_stem = X_q1_sw_stem_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_stem_d2v[i+1]
  arr_q1_sw_stem = np.vstack([arr_q1_sw_stem, arrT])

arr_q2_sw_stem = X_q2_sw_stem_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_stem_d2v[i+1]
  arr_q2_sw_stem = np.vstack([arr_q2_sw_stem, arrT])

assert arr_q1_sw_stem.shape == arr_q2_sw_stem.shape

# arr_q1_sw_stem = create_arrays(X_q1_sw_stem_d2v)
# arr_q2_sw_stem = create_arrays(X_q2_sw_stem_d2v)

In [ ]:
# Without stopwords, not lemmatized, not stemmed
size = len(df) - 1
arr_q1_out = X_q1_out_d2v[0] 

for i in range(size):
  arrT = X_q1_out_d2v[i+1]
  arr_q1_out = np.vstack([arr_q1_out, arrT])

arr_q2_out = X_q2_out_d2v[0] 

for i in range(size):
  arrT = X_q2_out_d2v[i+1]
  arr_q2_out = np.vstack([arr_q2_out, arrT])

assert arr_q1_out.shape == arr_q2_out.shape

In [ ]:
# With stopword, not lemmatized, not stemmed

size = len(df) - 1
arr_q1_sw = X_q1_sw_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_d2v[i+1]
  arr_q1_sw = np.vstack([arr_q1_sw, arrT])

arr_q2_sw = X_q2_sw_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_d2v[i+1]
  arr_q2_sw = np.vstack([arr_q2_sw, arrT])

assert arr_q1_sw.shape == arr_q2_sw.shape

### Convert to dataframe 

In [25]:
def df_arr(arr_q1, arr_q2, df):
  temp1 = pd.DataFrame(arr_q1)
  temp2 = pd.DataFrame(arr_q2)
  X_temp = pd.concat([temp1, temp2], axis=1)
  y = df['is_duplicate']
  y = np.array(y)
  y_df = pd.DataFrame(y)
  df_X = X_temp.join(y_df,  lsuffix='_left', rsuffix='_right')
  #X_temp['is_duplicate'] = df['is_duplicate']
  return df_X

In [ ]:
# Dataframe - Lemmatized without stopwords 
lemma_df = df_arr(arr_q1_lemma, arr_q2_lemma, df)

In [ ]:
# Dataframe - Lemmatized with stopwords 
sw_lemma_df = df_arr(arr_q1_sw_lemma, arr_q2_sw_lemma, df)

In [26]:
# Dataframe - Stemmed without stopwords 
stem_df = df_arr(arr_q1_stem, arr_q2_stem, df)

In [27]:
# Dataframe - Lemmatized with stopwords 
sw_stem_df = df_arr(arr_q1_sw_stem, arr_q2_sw_stem, df)

In [ ]:
# Dataframe - With stopwords, not lemmatized, not stemmed
sw_df = df_arr(arr_q1_sw, arr_q2_sw, df)

In [ ]:
# Dataframe - Without stopwords, not lemmatized, not stemmed
out_df = df_arr(arr_q1_out, arr_q2_out, df)

In [28]:
stem_df.head(2)


,0_left,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,0_right
0,-0.007656,-0.017189,-0.058634,0.072262,-0.014387,0.013050,-0.043178,-0.065979,0.055877,-0.036416,...,-0.054352,0.042411,-0.017314,-0.025635,0.013351,-0.065088,0.013830,0.062138,0.054103,0
1,-0.031230,0.023416,0.008874,0.074721,-0.016633,-0.025616,-0.050669,-0.018542,0.036259,-0.002108,...,-0.076787,0.021046,-0.026546,0.046404,-0.064159,0.009849,0.009103,0.022082,0.063962,1


In [29]:
df_stem.head(2)

,q1,q2,is_duplicate
11965,list job titl rsm audit firm kuwait,countri best work kuwait,0
342404,peopl 94 allow vote,peopl 97 allow vote,1


In [30]:
df_sw_stem.head(2)

,q1,q2,is_duplicate
11965,"[, what, is, the, list, of, job, titl, at, rsm...","[, which, countri, is, the, best, to, work, in...",0
342404,"[, should, peopl, over, 94, not, be, allow, to...","[, should, peopl, over, 97, not, be, allow, to...",1


### Split Data

In [ ]:
# Lemmatized with stopwords 
y_sw_lemma= sw_lemma_df['0_right']
X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma = train_test_split(sw_lemma_df, y_sw_lemma, stratify = y_sw_lemma, test_size = 0.2, random_state = 0)

In [ ]:
# Lemmatized without stopwords 
y_lemma= lemma_df['0_right']
X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma = train_test_split(lemma_df, y_lemma, stratify = y_lemma, test_size = 0.2, random_state = 0)

In [31]:
# Stemmed with stopwords 
y_sw_stem= sw_stem_df['0_right']
X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem = train_test_split(sw_stem_df, y_sw_stem, stratify = y_sw_stem, test_size = 0.2, random_state = 0)

In [32]:
# Stemmed without stopwords 
y_stem= stem_df['0_right']
X_train_stem, X_test_stem, y_train_stem, y_test_stem = train_test_split(stem_df, y_stem, stratify = y_stem, test_size = 0.2, random_state = 0)

In [ ]:
# With Stopwords, not lemmatized, not stemmed 
y_sw= sw_df['0_right']
X_train_sw, X_test_sw, y_train_sw, y_test_sw = train_test_split(sw_df, y_sw, stratify = y_sw, test_size = 0.2, random_state = 0)

In [ ]:
# Without Stopwords, not lemmatized, not stemmed 
y_out= out_df['0_right']
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(out_df, y_out, stratify = y_out, test_size = 0.2, random_state = 0)

# MODELS

### Random Forest

In [33]:
def rf_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Random Forest Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    rf_param_grid = {'n_estimators':[20, 50, 80], 'min_samples_split':[3, 13], 'max_depth': [10, 30, None]}
    rf_clf = RandomForestClassifier(random_state = 0)
    
    rf_search = HalvingGridSearchCV(rf_clf, rf_param_grid, cv = splits, factor = 2, scoring = 'accuracy', 
                                        max_resources = 5, aggressive_elimination = False, verbose = 0)
    rf_search.fit(X_train, y_train)
    
    rf_model = rf_search.best_estimator_
    
    y_pred = rf_model.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    print (rf_model)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, rf_model

## Logistic Regression

In [34]:
def lr_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Logistic Regression Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    lr_param_grid = {'C':[100, 10, 1.0, 0.1, 0.01], 'solver':['sag', 'lbfgs']}
    lr_clf = LogisticRegression(penalty = 'l2', random_state = 0)
   
    lr_d2v_search = HalvingGridSearchCV(lr_clf, lr_param_grid, cv = splits, factor = 2, scoring = 'accuracy', verbose = 0)
    lr_d2v_search.fit(X_train, y_train)
    
    lr_d2v_model = lr_d2v_search.best_estimator_
    y_pred_lr_d2v = lr_d2v_model.predict(X_test)
    lr_loss_d2v = metrics.log_loss(y_test, y_pred_lr_d2v)
    lr_acc_d2v = metrics.accuracy_score(y_test, y_pred_lr_d2v)
    print (lr_d2v_model)
    print('log loss', lr_loss_d2v, '\nacc', lr_acc_d2v)
    return lr_acc_d2v, lr_loss_d2v, y_pred_lr_d2v   

# Support Vector Classification - SVC

In [37]:
from sklearn.svm import SVC
def svc_clf(X_train, X_test, y_train, y_test):
  clf_svc = SVC(C=1.0, kernel='rbf', degree=3, max_iter=-1, decision_function_shape='ovr', random_state=None)   #kernel default='rbf', decision_function_shape='ovr', random_state=None


  clf_svc.fit(X_train, y_train)

  y_pred = clf_svc.predict(X_test)
  log_loss = metrics.log_loss(y_test, y_pred)
  accuracy = metrics.accuracy_score(y_test, y_pred)
  precision = metrics.precision_score(y_test, y_pred)
  recall = metrics.recall_score(y_test, y_pred)
  f1_score = metrics.f1_score(y_test, y_pred)
  return accuracy, precision, recall, f1_score, log_loss, y_pred

# Activate model functions

#### On Lemmatized data

In [ ]:
# Lemmatized, without stopwords

#Random Forest
acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma, y_pred_rf_lemma, berf_lemma  = rf_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_rf_lemma = classification_report(y_pred_rf_lemma, y_test_lemma)
matrix_rf_lemma = confusion_matrix(y_pred_rf_lemma, y_test_lemma)
print('Classification Report - RF\n', report_rf_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_lemma)

#Logistic Regression
lr_acc_lemma, lr_loss_lemma, y_pred_lr_lemma = lr_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)
print()

# SVC 
acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma = svc_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_svc_lemma = classification_report(y_pred_svc_lemma, y_test_lemma)
matrix_svc_lemma = confusion_matrix(y_pred_svc_lemma, y_test_lemma)

print('Classification Report - SVC\n', report_svc_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_lemma)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      0.97      0.99       639
           1       0.96      1.00      0.98       361

    accuracy                           0.98      1000
   macro avg       0.98      0.99      0.98      1000
weighted avg       0.98      0.98      0.98      1000


Confusion Matrix - RF
 [[622  17]
 [  0 361]]
LogisticRegression(C=0.1, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       622
           1       1.00      1.00      1.00       378

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000


Confusion Matrix - SVC
 [[622 

In [ ]:
# Lemmatized, with stopwords

#Random Forest
acc_rf_sw_lemma, precision_rf_sw_lemma, recall_rf_sw_lemma, f1_score_rf_sw_lemma, log_loss_rf_sw_lemma, y_pred_rf_sw_lemma, berf_sw_lemma  = rf_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)

report_rf_sw_lemma = classification_report(y_pred_rf_sw_lemma, y_test_sw_lemma)
matrix_rf_sw_lemma = confusion_matrix(y_pred_rf_sw_lemma, y_test_sw_lemma)
print('Classification Report - RF\n', report_rf_sw_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw_lemma)
print()

#Logistic Regression
lr_acc_sw_lemma, lr_loss_sw_lemma, y_pred_lr_sw_lemma = lr_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)
print()

# SVC 
acc_svc_sw_lemma, precision_svc_sw_lemma, recall_svc_sw_lemma, f1_score_svc_sw_lemma, log_loss_svc_sw_lemma, y_pred_svc_sw_lemma = svc_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)

report_svc_sw_lemma = classification_report(y_pred_svc_sw_lemma, y_test_sw_lemma)
matrix_svc_sw_lemma = confusion_matrix(y_pred_svc_sw_lemma, y_test_sw_lemma)

print('Classification Report - SVC\n', report_svc_sw_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw_lemma)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      0.96      0.98       656
           1       0.93      1.00      0.96       344

    accuracy                           0.97      1000
   macro avg       0.96      0.98      0.97      1000
weighted avg       0.98      0.97      0.97      1000


Confusion Matrix - RF
 [[630  26]
 [  0 344]]
LogisticRegression(C=0.1, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       629
           1       1.00      0.99      1.00       371

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000


Confusion Matrix - SVC
 [[628 

# With stopwords, not lemmatized nor stemmed

In [ ]:
# With stopwords, not lemmatized, nor stemmed

#Random Forest
acc_rf_sw, precision_rf_sw, recall_rf_sw, f1_score_rf_sw, log_loss_rf_sw, y_pred_rf_sw, berf_sw  = rf_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)

report_rf_sw = classification_report(y_pred_rf_sw, y_test_sw)
matrix_rf_sw = confusion_matrix(y_pred_rf_sw, y_test_sw)
print('Classification Report - RF\n', report_rf_sw)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw)
print()

#Logistic Regression
lr_acc_sw, lr_loss_sw, y_pred_lr_sw = lr_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)
print()

# SVC 
acc_svc_sw, precision_svc_sw, recall_svc_sw, f1_score_svc_sw, log_loss_svc_sw, y_pred_svc_sw = svc_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)

report_svc_sw = classification_report(y_pred_svc_sw, y_test_sw)
matrix_svc_sw = confusion_matrix(y_pred_svc_sw, y_test_sw)

print('Classification Report - SVC\n', report_svc_sw)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      0.99      1.00       634
           1       0.99      1.00      0.99       366

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000


Confusion Matrix - RF
 [[629   5]
 [  1 365]]

LogisticRegression(C=0.1, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0



In [ ]:
# SVC 
acc_svc_sw, precision_svc_sw, recall_svc_sw, f1_score_svc_sw, log_loss_svc_sw, y_pred_svc_sw = svc_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)

report_svc_sw = classification_report(y_pred_svc_sw, y_test_sw)
matrix_svc_sw = confusion_matrix(y_pred_svc_sw, y_test_sw)

print('Classification Report - SVC\n', report_svc_sw)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw)

Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       630
           1       1.00      1.00      1.00       370

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000


Confusion Matrix - SVC
 [[630   0]
 [  0 370]]


### Without stopwords, not lemmatized nor stemmed

In [ ]:
# Without stopwords, not lemmatized, nor stemmed

#Random Forest
acc_rf_out, precision_rf_out, recall_rf_out, f1_score_rf_out, log_loss_rf_out, y_pred_rf_out, berf_out  = rf_clf(X_train_out, X_test_out, y_train_out, y_test_out)

report_rf_out = classification_report(y_pred_rf_out, y_test_out)
matrix_rf_out = confusion_matrix(y_pred_rf_out, y_test_out)
print('Classification Report - RF\n', report_rf_out)
print()
print('Confusion Matrix - RF\n', matrix_rf_out)
print()

#Logistic Regression
lr_acc_out, lr_loss_out, y_pred_lr_out = lr_clf(X_train_out, X_test_out, y_train_out, y_test_out)
print()

# SVC 
acc_svc_out, precision_svc_out, recall_svc_out, f1_score_svc_out, log_loss_svc_out, y_pred_svc_out = svc_clf(X_train_out, X_test_out, y_train_out, y_test_out)

report_svc_out = classification_report(y_pred_svc_out, y_test_out)
matrix_svc_out = confusion_matrix(y_pred_svc_out, y_test_out)

print('Classification Report - SVC\n', report_svc_out)
print()
print('Confusion Matrix - SVC\n', matrix_svc_out)

# Stemmed data with stopwords

In [40]:
# Stemmed with stopwords

#Random Forest
acc_rf_sw_stem, precision_rf_sw_stem, recall_rf_sw_stem, f1_score_rf_sw_stem, log_loss_rf_sw_stem, y_pred_rf_sw_stem, berf_sw_stem  = rf_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)

report_rf_sw_stem = classification_report(y_pred_rf_sw_stem, y_test_sw_stem)
matrix_rf_sw_stem = confusion_matrix(y_pred_rf_sw_stem, y_test_sw_stem)
print('Classification Report - RF\n', report_rf_sw_stem)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw_stem)
print()

#Logistic Regression
lr_acc_sw_stem, lr_loss_sw_stem, y_pred_lr_sw_stem = lr_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)
print()

# SVC 
acc_svc_sw_stem, precision_svc_sw_stem, recall_svc_sw_stem, f1_score_svc_sw_stem, log_loss_svc_sw_stem, y_pred_svc_sw_stem = svc_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)

report_svc_sw_stem = classification_report(y_pred_svc_sw_stem, y_test_sw_stem)
matrix_svc_sw_stem = confusion_matrix(y_pred_svc_sw_stem, y_test_sw_stem)

print('Classification Report - SVC\n', report_svc_sw_stem)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw_stem)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.99      0.94      0.96       660
           1       0.90      0.98      0.94       340

    accuracy                           0.95      1000
   macro avg       0.94      0.96      0.95      1000
weighted avg       0.96      0.95      0.95      1000


Confusion Matrix - RF
 [[621  39]
 [  7 333]]

LogisticRegression(C=0.01, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       628
           1       1.00      1.00      1.00       372

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000


Confusion Matrix - SVC
 [[62

In [38]:
# SVC 
acc_svc_sw_stem, precision_svc_sw_stem, recall_svc_sw_stem, f1_score_svc_sw_stem, log_loss_svc_sw_stem, y_pred_svc_sw_stem = svc_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)

report_svc_sw_stem = classification_report(y_pred_svc_sw_stem, y_test_sw_stem)
matrix_svc_sw_stem = confusion_matrix(y_pred_svc_sw_stem, y_test_sw_stem)

print('Classification Report - SVC\n', report_svc_sw_stem)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw_stem)

Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       628
           1       1.00      1.00      1.00       372

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000


Confusion Matrix - SVC
 [[628   0]
 [  0 372]]


# Stemmed without stopwords


In [41]:
# Stemmed with stopwords

#Random Forest
acc_rf_stem, precision_rf_stem, recall_rf_stem, f1_score_rf_stem, log_loss_rf_stem, y_pred_rf_stem, berf_stem  = rf_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)

report_rf_stem = classification_report(y_pred_rf_stem, y_test_stem)
matrix_rf_stem = confusion_matrix(y_pred_rf_stem, y_test_stem)
print('Classification Report - RF\n', report_rf_stem)
print()
print('Confusion Matrix - RF\n', matrix_rf_stem)
print()

#Logistic Regression
lr_acc_stem, lr_loss_stem, y_pred_lr_stem = lr_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)
print()

# SVC 
acc_svc_stem, precision_svc_stem, recall_svc_stem, f1_score_svc_stem, log_loss_svc_stem, y_pred_svc_stem = svc_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)

report_svc_stem = classification_report(y_pred_svc_stem, y_test_stem)
matrix_svc_stem = confusion_matrix(y_pred_svc_stem, y_test_stem)

print('Classification Report - SVC\n', report_svc_stem)
print()
print('Confusion Matrix - SVC\n', matrix_svc_stem)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       631
           1       0.99      1.00      1.00       369

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000


Confusion Matrix - RF
 [[628   3]
 [  0 369]]

LogisticRegression(C=0.1, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       628
           1       1.00      1.00      1.00       372

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000


Confusion Matrix - SVC
 [[628

In [39]:
# SVC 
acc_svc_stem, precision_svc_stem, recall_svc_stem, f1_score_svc_stem, log_loss_svc_stem, y_pred_svc_stem = svc_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)

report_svc_stem = classification_report(y_pred_svc_stem, y_test_stem)
matrix_svc_stem = confusion_matrix(y_pred_svc_stem, y_test_stem)

print('Classification Report - SVC\n', report_svc_stem)
print()
print('Confusion Matrix - SVC\n', matrix_svc_stem)

Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       628
           1       1.00      1.00      1.00       372

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000


Confusion Matrix - SVC
 [[628   0]
 [  0 372]]


# Doc2Vec and similarity

In [ ]:
model_doc2vec = Doc2Vec(vector_size =20, window=2, min_count=1, epochs=50)

In [ ]:
model_doc2vec.build_vocab(X_train_tagged)

In [ ]:
model_doc2vec.train(X_train_tagged, total_examples=model_doc2vec.corpus_count, epochs=model_doc2vec.epochs)

In [ ]:
print(model_doc2vec.infer_vector(X_sw_test))

[ 0.00553507 -0.02057612 -0.00601494  0.01590716  0.00477619 -0.00957217
 -0.01354616  0.01898067 -0.00717559  0.00853975  0.00150559 -0.01450056
 -0.00628278 -0.02179538 -0.01993607 -0.02084691  0.00312932 -0.0190215
 -0.00581192  0.01894295]


In [ ]:
#TEST THE MODEL ON X_sw_test

test = model_doc2vec.infer_vector(X_sw_test)
model_doc2vec.docvecs.most_similar(positive = [test])
#positive = List of sentences that contribute positively.


[(1607, 0.3674626350402832),
 (1454, 0.3508557975292206),
 (1876, 0.3459669351577759),
 (2316, 0.3180397152900696),
 (1868, 0.31006860733032227),
 (389, 0.3054264485836029),
 (1438, 0.28425800800323486),
 (1760, 0.272051066160202),
 (2109, 0.2675776481628418),
 (2054, 0.2670101821422577)]

# buen codigo para doc2vec cosine similarity

In [ ]:
# import
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
q1_tag = [TaggedDocument(d, [i]) for i, d in enumerate(df_clean['q1'])]
q2_tag = [TaggedDocument(d, [i]) for i, d in enumerate(df_clean['q2'])]
#tagged_data

In [ ]:
q1_model = Doc2Vec(q1_tag, vector_size = 20, window = 2, min_count = 1, epochs = 100)
q2_model = Doc2Vec(q2_tag, vector_size = 20, window = 2, min_count = 1, epochs = 100)

In [ ]:
q1_model[0]

array([ 0.5810266 ,  1.2083697 ,  0.9162264 , -0.0174998 , -0.15741944,
       -1.4726185 , -0.62223524,  0.5812599 ,  0.7438206 , -0.04712546,
       -0.8470433 ,  0.6188074 ,  0.6834843 ,  0.34443977, -0.66070366,
        0.66500264,  0.1121475 , -0.8982408 ,  0.18651539, -0.18247873],
      dtype=float32)

In [ ]:
# #cosine similarity - to finish 
# test_doc = word_tokenize("What is the step by step guide to invest".lower())
# test_doc_vector = model.infer_vector(test_doc)
# model.docvecs.most_similar(positive = [test_doc_vector])


[(1555, 0.7962811589241028),
 (2760, 0.7377956509590149),
 (1810, 0.7373658418655396),
 (2765, 0.7061798572540283),
 (2729, 0.6963450312614441),
 (1806, 0.688534677028656),
 (1839, 0.685553789138794),
 (2233, 0.6750451326370239),
 (1854, 0.6687722206115723),
 (2190, 0.6662200093269348)]

buen codigo

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
q1_lemma = [TaggedDocument(d, [i]) for i, d in enumerate(df_lemma['q1'])]
q2_lemma = [TaggedDocument(d, [i]) for i, d in enumerate(df_lemma['q2'])]


In [ ]:
model = Doc2Vec(tagged_data_lemma, vector_size = 10, window = 2, min_count = 1, epochs = 100)
model

In [ ]:
# test_doc = word_tokenize("What is the step by step guide to invest".lower())
# test_doc_vector = model.infer_vector(test_doc)
# model.docvecs.most_similar(positive = [test_doc_vector])

[(1854, 0.8070089221000671),
 (42, 0.7843912839889526),
 (958, 0.779266357421875),
 (1183, 0.7622733116149902),
 (1839, 0.7594348192214966),
 (2007, 0.7579702138900757),
 (2760, 0.7547283172607422),
 (909, 0.7446797490119934),
 (2765, 0.7345895767211914),
 (1835, 0.732286810874939)]

In [ ]:
y= df['0_right']
X_train, X_test, y_train, y_test = train_test_split(df, y, stratify = y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
#X, y = make_classification(random_state=0, shuffle=False)
#clf = RandomForestClassifier(max_depth=2, random_state=0)
#clf.fit(X_train, y_train)


In [ ]:
from sklearn.model_selection import ShuffleSplit

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

In [ ]:
def rf_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Random Forest Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    rf_param_grid = {'n_estimators':[200, 500, 800], 'min_samples_split':[5, 15], 'max_depth': [70, 150, None]}
    rf_clf = RandomForestClassifier(random_state = 0)
    
    rf_bow_search = HalvingGridSearchCV(rf_clf, rf_param_grid, cv = splits, factor = 2, scoring = 'accuracy', 
                                        max_resources = 40, aggressive_elimination = True,verbose = 0)
    rf_bow_search.fit(X_train, y_train)
    
    rf_bow_model = rf_bow_search.best_estimator_
    
    y_pred = rf_bow_model.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    print (rf_bow_model)
    #print('log loss', rf_loss_bow, '\nacc', rf_acc_bow)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, rf_bow_model

In [ ]:
acc_rf, precision_rf, recall_rf, f1_score_rf, log_loss_rf, y_pred_rf, berf  = rf_clf(X_train, X_test, y_train, y_test)

RandomForestClassifier(min_samples_split=15, n_estimators=500, random_state=0)


In [ ]:
report_BoW_rf = classification_report(y_pred_rf, y_test)
matrix_BoW_rf = confusion_matrix(y_pred_rf, y_test)
print('Classification Report - RF\n', report_BoW_rf)
print()
print('Confusion Matrix - RF\n', matrix_BoW_rf)

Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       381
           1       1.00      1.00      1.00       219

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600


Confusion Matrix - RF
 [[381   0]
 [  0 219]]


# SCV

In [ ]:
from sklearn.svm import SVC
def svc_clf(X_train, X_test, y_train, y_test):
  clf_svc = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200
              , class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)


  clf_svc.fit(X_train, y_train)

  y_pred = clf_svc.predict(X_test)
  log_loss = metrics.log_loss(y_test_lemma, y_pred)
  accuracy = metrics.accuracy_score(y_test, y_pred)
  precision = metrics.precision_score(y_test, y_pred)
  recall = metrics.recall_score(y_test, y_pred)
  f1_score = metrics.f1_score(y_test, y_pred)
  return accuracy, precision, recall, f1_score, log_loss, y_pred


In [ ]:
acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, fq_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma = svc_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_svc_lemma = classification_report(y_pred_svc_lemma, y_test_lemma)
matrix_svc_lemma = confusion_matrix(y_pred_svc_lemma, y_test_lemma)

print('Classification Report - SVC\n', report_svc_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_lemma)